<img src='./img/hhn.png' width="25%" align="right" />

<img src='./img/bar.png' width="100%" align="center" />

# Data Science Grundlagen

## Methodensteckbrief LightGBM

### Autoren 

Alexander Voss (208384), Fabian Reichert (194205), Osman Toptik (192327)

### Installation LightGBM

Da LightGBM nicht in der Bibliothek Scikit-Learn enthalten ist muss dies gesondert installiert werden. Um LightGBM zu installieren kann in einer Kommandozeile vom Jupyter Notebook der Installationsbefehl: 

! pip install lightbm 

ausgeführt werden um die benötigte Bibliothek zu installieren.

## Einführung

LightGBM wurde durch Microsoft im Jahr 2017 entwickelt und ist ein Open-Source-Framework für maschinelles Lernen. Im Gegensatz zu anderen Boosting-Algorithmen verwendet LightGBM für das Training von Entscheidungsbäumen Histogramm basierte Algorithmen, die die Zeitkomplexität reduzieren können [14].

Der Gradient Boosting Decision Tree (GBDT) ist ein populärer Algorithmus für maschinelles Lernen und verfügt über eine Reihe von effektiven Implementierungen wie XGBoost und pGBRT. GBDT zeichnet sich  durch seine Effizienz, Genauigkeit und Interpretierbarkeit aus. Der GBDT erzielt bei vielen maschinellen Lernaufgaben Leistungen auf dem neuesten Stand der Technik, wie z.B. Mehrklassen Klassifizierung, Klick Vorhersage und Lern to rank. In den letzten Jahren, mit dem Aufkommen großer Datenmengen (sowohl in Bezug auf die Anzahl der Features als auch auf die Anzahl der Instanzen), steht GBDT vor neuen Herausforderungen, insbesondere im Hinblick auf den Kompromiss zwischen Genauigkeit und Effizienz. Herkömmliche GBDT-Implementierungen müssen für jedes Merkmal alle Instanzen der Daten scannen, um den Informationsgewinn aller möglichen Aufteilungspunkte abschätzen zu können. Dementsprechend ist die Berechnungskomplexität des GBDT proportional, sowohl im Bezug zu Anzahl der Merkmale als auch zur Anzahl der Instanzen. Bei der Verarbeitung großer Datenmengen wird diese Implementierung  sehr zeitaufwändig. 

Um diese Herausforderung zu lösen, besteht die Idee darin, die Anzahl der Dateninstanzen und der Merkmale zu reduzieren. Dies erweist sich jedoch als schwierig. Beispielsweise ist so unklar, wie die Datenerfassung für GBDT am besten funktioniert. Es gibt verschiedene Arbeiten, die Daten stichprobenartig nach ihren Gewichtungen erfassen, um den Trainingsprozess des Boostings zu beschleunigen, diese können jedoch nicht direkt auf GBDT angewendet werden, da es keine Stichprobengewichtung im GBDT gibt. Folgende zwei Techniken werden zur Lösung vorgeschlagen [5].

## Aufbau Basisverständnis

LightGBM zählt mit der Vorstellung im Jahr 2017 zu einem der neusten Gradient Boosting Modellen. Das Modell fokussiert die Steigerung der Effizienz und Skalierbarkeit im Kontext großer und hochdimensionaler Datenmengen [5].

### Gradient Descent

Das Gradientenverfahren ist ein Optimierungsansatz ein lokales oder globales Minimum einer Kostenfunktion zu finden.

<img src="img/Gradient_descent_method.png" alt="Drawing" style="width: 350px;"/>
<center>Abbildung 1. Beispiel Gradient Descent Methode [12]</center>

Dem Minimum wird sich schrittweise anhand einer "Learning rate" angenähert. Mit der Wahl zu großer Schritte besteht die Gefahr, dass das Minimum nicht gefunden wird. Kleine Schritte führen hingehen zu einem präzieserem Ergebnis, benötigen jedoch mehr Rechenkapazitäten.

<img src="img/gradient-descent-learning-rate.png" alt="Drawing" style="width: 350px;"/>
<center>Abbildung 2. Unterschied zwische steiler und niedriger Learing rate [2]</center>

### Boosting

Boosting als Ensemblem Technik bedeutet, dass in sequentieller Abfolge und durch Kombination von sogenannten Weak Learner am Ende des Prozesses ein Strong Learner ensteht. Das gilt die Annahme, dass jeder Weak Learner ein besseres Ergebnis erzielt als eine zufällige Vermutung ("Raten"). So lernt der nachfolgende Weak Learner von dem vorherigen, indem dieser den Fokus auf falsch klassifizierte Dateninstanzen legt [15].

### Gradient Boosting Decision Tree

Beim GBDT als Methode des Ensemble Learing werden Entscheidungsbäume in sequentieller Reihenfolge trainiert. Hierbei werden die nachfolgenden Bäume mit Hilfe der Residuen angepasst. Für die Vebesserung der Vorhersagen, wird die Differenz zwischen der aktuellen Annäherung und dem bekannten korrekten Zielvektor, dem sogenannten Residuum, betrachtet. Dann wird ein schwaches Modell trainiert, das den Merkmalsvektor auf diesen Residuenvektor abbildet. Durch Hinzufügen eines von einem schwachen Modell vorhergesagten Residuums zur Approximation eines vorhandenen Modells wird das Modell in Richtung des korrekten Zielvektors verschoben, wodurch das Modell iterati stärker und präzisier wird. [12,10].

## Funktionsweise

LightGBM behinhaltet zwei besondere Herangehensweisen, um die Nachteile von oftmals langen Berechungszeiten zu adressieren.

### Allgemein


Im Gegesatz zu Algorithmen, wie bei XGBoost, deren Entscheidungsbäume horizontal wachsen, setzt LightGBM auf vertikales Wachstum. Man unterscheidet demnach blattweises (leaf-wise) und ebenweises (level-wise) Wachstum. Um zu wachsen, wählt LightGBM das Blatt aus, mit dem höchsten Informationsgewinn, statt in einer festen Reihenfolge zu erweitern. Wenn das gleiche Blatt wächst, kann der Algorithmus für blattweises Wachstum im Vergleich zu einem Algorithmus für ebenenweises die Präzision verbessern [1].

<img src="img/level-wise.png" alt="Drawing" style="width:100%"/> | <img src="img/leaf-wise.png" alt="" style="width:100%"/>
- | - 
<center>Abbildung 3. level-wise decision tree [7]</center> | <center>Abbildung 4. leaf-wise cesision tree [7]</center>

Der iterative Ansatz von LightGBM als Boosting Algorithmus trägt durch die blattweise Ausprägung des Entscheidungsbaus dazu bei, dass je Iteration der nächste Weak Learner durch die Erkenntnisse vom vorhergehenden angepasst wird und in Summe ein Strong Leaner mit hoher Präzision entsteht.

### Gradient-based One-Side Sampling (GOSS)

GOSS bildet die Grundlage zur Reduzierung der Datenmenge, ohne dabei einen nennenswerten Präzisionsverlust hervorzurufen. Es wird angenommen, dass verschiedene Dateninstanzen unterschiedliche Rollen bei der Berechnung des Informationsgewinns spielen. 

**Hinweis**: Ein Gradient beschreibt die Änderung von einer Funktion, welche von mehreren Variablen (Parameter) abhängig ist, mit der Veränderung eines einzelnen Parameters.

- Kleiner Gradient: Instanzen mit kleinen Gradienten sind gereits gut trainiert (kleiner Trainingsfehler), wodurch der erzielbare Informationsgewinn niedrig ist. Auf der Abbildung 1 befinden sich kleinen Gradienten nahe dem Minimum der Kostenfunktion, sie beschreiben weniger steile Punkte und haben nur eine schwache Steigung in der Funktion. Somit kann aus diesen nur ein geringer Informationsgewinn gezogen werden.

- Großer Gradient: Instanzen mit großen Gradienten sind tendnenziell schlecht trainiert und der verbundene Fehler auf den Trainigsdaten ist groß. Auf der Kostfunktion beschreiben große Gradienten steiler Punkte, sodass diese zu einem größerem Informationsgewinn beitragen.

Aufgrund dieser Annahmen werden alle Instanzen entsprechenden ihres Gradienten absteigend sortiert, anschließend von oben a x 100% der Instanzen ausgewählt und von den restlichen Instanzen eine Zufallsstichprobe der Größe b x 100% gezogen. Die Stichprobe wird im letzten Schritt für die Berechnung des Informationsgewinn mit einer Konstanten gleich (1-a) / b multipliziert [5].

<img src="img/GOSS_Visualization_2.png" alt="Drawing" style="width: 500px;"/>
<center>Abbildung 5. Beispiel GOSS [13]</center>

Die Namensgebnung lässt sich durch die Herangehensweise der Technik ableiten. So werden alle Instanzen mit großen Gradienten bis zu einem bestimmten Schwellenwert behalten. Von den restlichen Daten werden ausschließlich Stichproben gezogen, dies bezeichnet man als "One-Side Sampling".

### Exclusive Feature Bundling (EFB)

Datensätze mit einer großen Anzahl von Merkmalen weisen oftmals nur spärliche Beobachtungen des jeweiligen Merkmals (Sparsity) auf, d.h. im Datensatz sind viele Nullwerte enthalten. Diese spärlichen Merkmale schließen sich normalerweise gegenseitig aus, was bedeutet, dass sie nicht gleichzeitig Werte ungleich Null annehmen und somit zu einem einzigen Merkmal, in sogenannten "discrete bins" zusammengefasst werden können [5,11].

<img src="img/EFB_Visualization.png" alt="Drawing" style="width: 350px;"/> 
<center>Abbildung 6. Beispiel EFB [6]</center>

Es ist erkennbar, dass Beobachtungen von den Features F1, F2 und F3 sich jeweils gegenseitig ausschließen und zusammengeführt werden könnne. Es entsteht eine **Dimensionsreduzierung** von O(#data × #feature) zu O(#data × #bundle), wobei die Anzahl der #bundle signifikant kleiner ist als die Anzahl der #feature. Somit wird die Anzahl der Merkmale effektiv reduziert, ohne das die Genauigkeit der Splitpunkt-bestimmung stark beeinträchtigt wird [5].

Der neue GBDT-Algorithmus mit den Techniken GOSS und EFB wird LightGBM genannt. Experimente an mehreren öffentlichen Datensätzen zeigen, dass LightGBM den Trainingsprozess um bis zu mehr als das 20-fache beschleunigen kann und dabei fast die gleiche Genauigkeit erreicht [5].

## Anwendungsvorraussetzungen

LightGBM setzt aufgrund seiner Implementierung auf der Basis eines Histogram-Algorithmus nummerische Daten für die Vararbeitung voraus. Viel mehr muss es sich um einen Subtypen numerischer Daten, nämlich um konitnuierliche Daten handeln. Diese können alle möglichen Werte annehmen und werden durch Messungen erhoben.

Im Falle, dass kategorische Daten vorliegen, dessen Merkmale also eine begrenzte Anzahl von Ausprägungen (Kategorien) haben, müssen diese vor der Verwendung in LightGBM umgewandelt werden. 

| kategorische Daten | konitnuierliche Daten |
| :- | :- |
| Personen = Nationalität (deutsch, finnisch, italienisch), Geschlecht (M / W) | Körpergroße, Gewicht, Blutdruck|

In der Datenaufbereitungsphase müssen die kategorische Daten durch Umwandlungsmehtoden bearbeitet werden. Denn einzelenen kategorischen Variablen werden nummerische Werte zugewiesen, so wird aus "deutsch" = 0, "finnsch" = 1, usw..


<img src="img/categorical_vs_continuous-data.jpg" alt="" style="width: 500px;"/>
<center>Abbildung 7. Codieren von kategorischen Daten [11]</center>

#### Label-encoding /  Integer-encoding

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
data = {'Geschlecht':  ['weiblich', 'männlich ','weiblich','weiblich', 'männlich '],
        'Nationalität': ['deutsch', 'finnisch','deutsch', 'finnisch', 'italienisch'],
        }
df = pd.DataFrame (data, columns = ['Geschlecht','Nationalität'])
print('Before label encoding: ')
print(df)

df['Geschlecht'] = LabelEncoder().fit_transform(df['Geschlecht'])
df['Nationalität'] = LabelEncoder().fit_transform(df['Nationalität'])
print()
print('After label encoding: ')
print(df)

Before label encoding: 
  Geschlecht Nationalität
0   weiblich      deutsch
1  männlich      finnisch
2   weiblich      deutsch
3   weiblich     finnisch
4  männlich   italienisch

After label encoding: 
   Geschlecht  Nationalität
0           1             0
1           0             1
2           1             0
3           1             1
4           0             2


## Stärken

LightGBM basiert auf einem Histogramm-Algorithmus, welcher es ermöglich die Häufigkeit bestimmter Merkmale zusammenzufassen und daraufbasiernd zur Entscheidungsfindung mit Hilfe von Entscheidungsbäumen beizutragen. Die Erststellung der Merkmalshistogramme während der Modellierung ist gilt als einer der Kernpunkte, deren Berechnung sehr ressourcen- und zeitintensiv ist [8]. LightGBM bietet die Möglichkeit, für die Beschleunigung dieses Prozesses und die Erstellung des Entscheidungsbaums, den Grafikprozessor (GPU) zu nutzen [4].

LightGBM steht für die effiziente Speichernutzung, da kategorische Daten vor der Nutzung zu erst codieren werden müssen. Durch die Verwendung konitnuierlicher Daten, ist es sich nicht Notwednig für speicherintensive Zeichenketten Ressourcen nutzen zu müssen. Ebenso trägt die Zusammenfassung exklusiver Merkmale in diskreten Klassen ("Bins") dazu bei, dass der benötigte Speicherplatz minimal gehalten werden kann.

Durch die rapide Generierung von Daten ist es, neben dem  Einsatz eines performaten Frameworks, ebenso vorteilhaft Prozesse parallel bearbeiten zu können. LightGBM unterstütz drei Möglichkeiten des "Parallel Learning" [9].
- Feature Parallel: Ziel das Finden des besten Splitpunktes eines Merkmals zu parallelisieren. 
- Data Parallel: Ziel die Entscheidungsfindung als solches zu parallelisieren.
- Voting Parallel
    
Durch den Einsatz von GOSS und EFB werden die spärlichenen Daten effizient zuammengefasst und effektiv die Datenmenge verringert. Aus diesem Grund gilt LightGBM vorallem für den Einsatz im Zusammenhang mit großen Datenmengen als sehr leistungsstark und zeichnet sich durch eine schnelle Berechung der Trainigsdaten aus.

## Schwächen

Zur gleichen Zeit ergibt sich jedoch durch die Reduzierung spärlicher Daten, durch die geannten Mehthoden, bei einer inital geringen Datenmenge ein gengensätzliches Bild. So werden durch die Reduzierung und Zusammenfassung der vorliegneden Merkmale, unter Umständen wichtige Informationen herausgefiltert. Dieses Vorgehen bei kleinen Datenmengen könnte sich negativ auf die Präzision und Informationsgewinn auswirken. Daran schließt sich an, dass die eingsetzte EFB Technik das volle Potential erst im Kontext hochdimensionaler Daten ausschöpfen kann. Bei hochdimensionalen Daten handeles sich um Daten deren Anzahl von Merkmalen p größer als die Anzahl der Beobachtungen N ist (p ≫ N) [3].

Die Autoren erwähnen in ihrer Schlussbetrachtung den Punkt, dass für die weitere Entwicklung des Frameworks die Verbesserung der Leistung im Bezug auf exklusive Merkmale in Betracht gezogen werden soll. Daraus lässt sich womöglich eine Schwäche ableiten, die hinsichtlich dem Umgang mit der Bündelung von großen Mengen exklusiver Merkmale, unabhängig davon ob diese schwachbesetzt oder stark ausgeprägt sind, indentifizieren [5].

Zusätztlich besteht das Problem eine optimale Bündelungsstrategie für die Zusammenführung von Featuren zu exklusiven Bündeln zu finden. Dies wird als NP-hart bezeichnet und bedeute, dass es innerhalb der Polynomzeit nicht möglich ist eine exakte Lösung zu finden [5].

Wie bereits bekannt, wurde LightGBM dazu entwickelt die Nachteile herkömmlicher GBDT Implementierungen, nämlich lange Laufzeiten in deren Berechnung, zu lösen [5]. Durch den in sich komplexeren Algorithmus, bestehend aus GOSS und EFB, leitet sich ein vergleichsweise höher initialer Abstimmungsaufwand (Tuning) ab. Der bei LightGBM verwendete Algorithmus des blattweisen (leaf-wise und best-first Vorgehen) Baumwachstums neigt bei falscher Parameter-Konfiguration ebenfalls zur Überanpassung (Overfitting) [12].

## Referenzen

[1] Bakshi, Kirti (2018): LightGBM: A Light Gradient Boosting Machine, https://www.techleer.com/articles/489-lightgbm-a-light-gradient-boosting-machine/ (abgerufen am 21.11.2020)

[2] Donges, Niklas (2019): Gradient Descent: An Introduction to One of Machine Learning's Most Popular Algorithms, https://builtin.com/data-science/gradient-descent (abgerufen am 21.11.2020)

[3] Friedman, J., Hastie, T., & Tibshirani, R. (2001). The elements of statistical learning (Vol. 1, No. 10). New York: Springer series in statistics.

[4] Huan Zhang, Si Si, and Cho-Jui Hsieh. Gpu-acceleration for large-scale tree boosting. arXiv preprint arXiv:1706.08359, 2017 

[5] Ke, G., Meng, Q., Finley, T., Wang, T., Chen, W., Ma, W., ... & Liu, T. Y. (2017). Lightgbm: A highly efficient gradient boosting decision tree. In Advances in neural information processing systems (pp. 3146-3154).

[6]Kurt, Kübra: Matematiği ve Python Uygulamasıyla LightGBM, https://medium.com/kaveai/matematiği-ve-python-uygulamasıyla-lightgbm-hafif-gradyan-artırma-makinesi-18d2f12e7870 (abgerufen am 21.11.2020)

[7] Mandot, Pushkar (2018) What is LightGBM, How to implement it? How to fine tune the parameters?, https://medium.com/@pushkarmandot/https-medium-com-pushkarmandot-what-is-lightgbm-how-to-implement-it-how-to-fine-tune-the-parameters-60347819b7fc (abgerufen am 21.11.2020)

[8] Microsoft: GPU Tuning Guide and Performance Comparison, https://lightgbm.readthedocs.io/en/latest/GPU-Performance.html (abgerufen am 21.11.2020)

[9] Microsoft: Optimization in Parallel Learning https://lightgbm.readthedocs.io/en/latest/Features.html#optimization-in-parallel-learning (abgerufen am 21.11.2020)

[10] Parr, Terence and Howard, Jeremy: Gradient boosting: frequently asked questions, https://explained.ai/gradient-boosting/faq.html (abgerufen am 21.11.2020)

[11] Sethi, Alakh (2020): One-Hot Encoding vs. Label Encoding using Scikit-Learn, https://www.analyticsvidhya.com/blog/2020/03/one-hot-encoding-vs-label-encoding-using-scikit-learn/ (abgerufen am 21.11.2020)

[12] Sharma, Abihshek (2018): What makes LightGBM lightning fast?, https://towardsdatascience.com/what-makes-lightgbm-lightning-fast-a27cf0d9785e (abgerufen am 21.11.2020)

[13] Wang, Beverly: Gradient Tree Boosting: XGBoost vs. LightGBM vs. CatBoost (Part 2), https://medium.com/@beverly.wang0005/gradient-tree-boosting-xgboost-vs-lightgbm-vs-catboost-part-2-275525458968 (abgerufen am 21.11.2020)

[14] Wang, G., & Liu, Z. (2020). Android malware detection model based on lightgbm. In Recent Trends in Intelligent Computing, Communication and Devices (pp. 237-243). Springer, Singapore.

[15] Zhi-Hua Zhou. Ensemble methods: foundations and algorithms. CRC press, 2012.
